# Trial Module

- __author__ = 'BlackDChase,MR-TLL'
- __version__ = '0.3.0'
- [Imports](#Imports)
- [Hyperparameter](#Hyperparameters)
- [Training A3C](#Training-A3C)
- [Master Agent](#Master-Agent)
- [Train and Save](#Train-and-Save)
- [Testing Model](#Testing-Model)
- [Train](#Train)
- [Environment](#Environment)
- [Test](#Test)
- [Plotting](#Plotting)

# Imports

In [ ]:
from lstm import LSTM
from env import LSTMEnv as ENV
#from TempEnv import TempEnv as ENV
from Agent import GOD
import log
import sys
import torch
from torch import nn, multiprocessing, device, Tensor
import matplotlib.pyplot as plt

# Hyperparameters

In [ ]:
"""
Hyperparameters
"""
stateSize = 13
log.info(f"stateSize = {stateSize}")
keywords={
        "n":6,
        "e":500,
        "t":50,
        "a":5,
        "d":False,
        "alr":1e-3,
        "clr":1e-3,
    }
stateSize = 13
log.info(f"stateSize = {stateSize}")

"""
Making the action space
"""
n=int(keywords['a']//2)
if keywords['a']%2==0:
    actionSpace = [i for i in range(-n*2,n*2+1,2)]
    actionSpace.pop(len(actionSpace)//2)
else:
    actionSpace = [i/10 for i in range(-n*25,n*25+1,25)]
keywords['a']=actionSpace

In [ ]:
actionSpace

# Training A3C

# Master Agent

In [ ]:
god = GOD(
    stateSize=int(stateSize),
    actionSpace=keywords["a"],
    debug=keywords["d"],
    maxEpisode=int(keywords["e"]),
    nAgent=int(keywords["n"]),
    trajectoryLength=int(keywords["t"]),
    alr=keywords["alr"],
    clr=keywords["clr"]
)
print("Master Agent Made")
log.info("GOD inititated")
log.info(f"Action space: {actionSpace}")

# Train and Save

In [ ]:
god.train()
god.saveModel("../Saved_model")

# Testing Model

# Enviornment

In [ ]:
#"""# Using LSTM Model as ENV
output_size = 13
input_dim = output_size
hidden_dim = 128
layer_dim = 1
model = LSTM(output_size, input_dim, hidden_dim, layer_dim,debug=keywords["d"])
model.loadM("ENV_MODEL/lstm_model.pt")
log.info(f"LSTM Model = {model}")
#env=ENV(model,"../Dataset/13_columns.csv")
env=ENV(
    model=model,
    dataset_path="../Dataset/col13data1.csv",
    actionSpace=actionSpace,
    debug=keywords["d"],
)
env.reset()
print("Environment inititated")
"""
env = ENV(stateSize,actionSpace)
#"""
log.info("Environment inititated")

god.giveEnvironment(env)
log.info("Environment parsed, Boss inititated")

# Test

In [ ]:
"""
for 100 timesteps
"""
time=100
a3cStates = god.test(time=time)
normalStates = god.getNormalStates(time=time)
profit,profitA3C = god.compare(a3cState=a3cStates,normalState=normalStates)

# Plotting

In [ ]:
plt.figure(dpi=100)
plt.xlabel("Episode")
plt.ylabel("Scalled Profit")
plt.plot(profit,label='Profits without A3C')
plt.plot(profitA3C,label='Profits with A3C')
plt.legend()
plt.show()